In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

#import numpy as np # linear algebra
#import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

#import os
#for dirname, _, filenames in os.walk('/kaggle/input'):
#    for filename in filenames:
#        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# Loading Images

In [ ]:
# data can be found here: https://www.kaggle.com/andrewmvd/ocular-disease-recognition-odir5k

In [ ]:
import os
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.utils import get_file

In [ ]:
pip install openpyxl

In [ ]:
data_directory = '../input/ocular-disease-recognition-odir5k/ODIR-5K/ODIR-5K/data.xlsx'
data = pd.read_excel(data_directory)
data.head()

The diagnoses can be broken down into: 
* Normal (N),
* Diabetes (D),
* Glaucoma (G),
* Cataract (C),
* Age related Macular Degeneration (A),
* Hypertension (H),
* Pathological Myopia (M),
* Other diseases/abnormalities (O)

Looking at the full_df, I am a little bit confused. It doesn't seem like the 'target' column lines up with the N, D, G, C, A, H, M, and O columns. For example in ID 0, the target is [1, 0, 0, 0, 0, 0, 0, 0] but the diagnosis is cataract which I think should have been [0, 0, 0, 1, 0, 0, 0, 0]. Also, for ID 2, the exact same target is present [1, 0, 0, 0, 0, 0, 0, 0] but this time it is normal in both eyes. So the target column seems very unreliable. I think I'm better off just engineering my own target based on the N, D, G, C, A, H, M, and O columns. Also the labels seem off since patient ID 0 says 'N' even though this patient has a cataract. Even more reason to engineer my own features.

In [ ]:
full_df = pd.read_csv('/kaggle/input/ocular-disease-recognition-odir5k/full_df.csv')
full_df.head(10)

# Re-engineering the 'target'

In [ ]:
# I'm only going to keep the columns that are actually useful. I don't need the filepath, left/right keywords
# Also, the labels and target are unreliable, as I have shown previously so I'll drop those too and create my own
df = full_df[['ID','Patient Age','Patient Sex','N','D','G','C','A','H','M','O','filename']]

In [ ]:
target = pd.Series(dtype='object')
# this code iterates through every row in the dataframe and creates a list which contains the correct values for the target, when reading the actual columns
# then appends this list to a series which contains the target values
for i in range(df.shape[0]):
    target_list = []
    for col in ['N','D','G','C','A','H','M','O']:
        if (df.loc[i][col] == 1):
            target_list.append(1)
        else:
            target_list.append(0)
    d = {'target':target_list}
    input_df_row = pd.Series(data=d)
    target = target.append(input_df_row)
target.index = df.index

In [ ]:
target